In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/train_waymo.yaml"   # train_tdt4265.yaml  train_waymo.yaml

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2020-04-11 18:21:44,308 SSD INFO: Loaded configuration file configs/train_waymo.yaml
2020-04-11 18:21:44,309 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet'   #vgg   #'basic'
        PRETRAINED: True
        OUT_CHANNELS: [512,1024,2048,512, 256, 128]     #[128, 256, 512, 512, 256, 128]
        INPUT_CHANNELS: 3
        DEPTH: 50
        DROP_BLOCK: False
INPUT:
    IMAGE_SIZE: 300
DATASETS:
    TRAIN: ("waymo_train",)
    TEST: ("waymo_val", )
SOLVER:
    CHOICE: 'SGD'
    MAX_ITER:  53000
    LR_STEPS: [20000, 400000]
    GAMMA: 0.1
    BATCH_SIZE: 32
    LR: 1.e-2
OUTPUT_DIR: 'outputs/res50_new_waymo_sgd_1e_f2_new_aug'  
DATASET_DIR: "datasets"
2020-04-11 18:21:44,309 SSD INFO: Running with config:
DATASETS:
  TEST: ('waymo_val',)
  TRAIN: ('waymo_train',)
DATASET_DIR: datasets
DATA_LOADER:
  NUM_WORKERS: 8
  PIN_MEMORY: True
EVAL_STEP: 500
INPUT:
  IMAGE_SIZE: 300
  PIXEL_MEAN: [123, 117, 104]
LOG_STEP: 10
MODEL:
  BACKBONE:
    AFTER_TRAINED: False
    DE

In [ ]:
model = start_train(cfg)

resnet
Detector initialized. Total Number of params:  39.55M
Backbone number of parameters: 37.52M
SSD Head number of parameters: 2.03M
2020-04-11 18:21:48,120 SSD.trainer INFO: No checkpoint found.
17936
Dataset loaded. Subset: train, number of images: 14348
2020-04-11 18:21:49,585 SSD.trainer INFO: Start training ...
2020-04-11 18:22:14,051 SSD.trainer INFO: iter: 000010, lr: 0.00347, total_loss: 17.765 (17.765), reg_loss: 6.942 (6.942), cls_loss: 10.823 (10.823), time: 2.276 (2.276), eta: 1 day, 9:29:54, mem: 9881M
2020-04-11 18:22:25,717 SSD.trainer INFO: iter: 000020, lr: 0.00360, total_loss: 13.533 (15.649), reg_loss: 6.185 (6.564), cls_loss: 7.348 (9.085), time: 1.167 (1.721), eta: 1 day, 1:19:49, mem: 9881M
2020-04-11 18:22:37,077 SSD.trainer INFO: iter: 000030, lr: 0.00373, total_loss: 10.803 (14.034), reg_loss: 5.588 (6.239), cls_loss: 5.214 (7.795), time: 1.136 (1.526), eta: 22:27:19, mem: 9881M
2020-04-11 18:22:50,116 SSD.trainer INFO: iter: 000040, lr: 0.00387, total_loss:

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)